In [1]:
# --- PATH SETUP ---
import sys
from pathlib import Path

notebook_dir = Path().resolve()
project_root = notebook_dir.parent         # <- this is eirgrid-pipeline
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

print("Notebook directory:", notebook_dir)
print("Project root:", project_root)
print("src path:", src_path)

import os
print("src contents:", os.listdir(src_path))

# --- IMPORTS ---
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta

from prophet import Prophet

from warehouse.readings import (
    get_all_metrics_wide,
    get_metric_series,
    get_latest_complete_local_day,
)


Notebook directory: C:\Users\roryo\Documents\DATA-Portfolio\eirgrid-pipeline\notebooks
Project root: C:\Users\roryo\Documents\DATA-Portfolio\eirgrid-pipeline
src path: C:\Users\roryo\Documents\DATA-Portfolio\eirgrid-pipeline\src
src contents: ['dashboard', 'ingest', 'models', 'pipeline', 'transform', 'warehouse']


Importing plotly failed. Interactive plots will not work.


In [2]:
latest = get_latest_complete_local_day()
print("Latest complete local day:", latest)

df_wind = get_metric_series("wind_actual", latest - timedelta(days=29), latest)
print("Rows:", len(df_wind))
df_wind.head()


Latest complete local day: 2025-11-26
Rows: 192


,ts_utc,value,metric_id,region_id
0,2025-11-25 00:00:00+00:00,1581.0,1,1
1,2025-11-25 00:15:00+00:00,1563.0,1,1
2,2025-11-25 00:30:00+00:00,1549.0,1,1
3,2025-11-25 00:45:00+00:00,1466.0,1,1
4,2025-11-25 01:00:00+00:00,1442.0,1,1


In [3]:
# Start from df_wind (ts_utc, value, metric_id, region_id)

df_prophet = (
    df_wind[["ts_utc", "value"]]
      .rename(columns={"ts_utc": "ds", "value": "y"})
      .assign(ds=lambda d: d["ds"].dt.tz_convert(None))  # <-- remove timezone
      .sort_values("ds")
      .reset_index(drop=True)
)

df_prophet.head(), df_prophet.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      192 non-null    datetime64[ns]
 1   y       192 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.1 KB


(                   ds       y
 0 2025-11-25 00:00:00  1581.0
 1 2025-11-25 00:15:00  1563.0
 2 2025-11-25 00:30:00  1549.0
 3 2025-11-25 00:45:00  1466.0
 4 2025-11-25 01:00:00  1442.0,
 None)

In [4]:
m = Prophet(
    daily_seasonality=True,   # we *do* have strong daily pattern
    weekly_seasonality=True,  # also weekly pattern in general
    yearly_seasonality=False  # we don't have a year of data yet
)

m.fit(df_prophet)


17:08:49 - cmdstanpy - INFO - Chain [1] start processing
17:08:50 - cmdstanpy - INFO - Chain [1] done processing
